In [92]:
import pandas as pd

from datetime import datetime, timedelta
import json
geo_json_path = 'data/geo_data/geojson-counties-fips.json'
with open(geo_json_path,'r') as response:
    counties = json.load(response)

In [121]:
counties_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
states_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
county_data = pd.read_csv(counties_url,dtype={"fips": str})
state_data = pd.read_csv(states_url,dtype={"fips": str})
print("Received latest NYT Data")

Received latest NYT Data


In [122]:
blank = county_data[county_data.county == "New York City"].copy()

burrs = [36061, 36047, 36081, 36005, 36085]
nyc_pop = 8400000
for each in burrs:
    test = blank.copy()
    test.fips = each
    test.cases = test.cases.apply(lambda x: int(x/5))
    test.deaths = test.deaths.apply(lambda x: int(x/5))
    
    county_data = county_data.append(test)

In [99]:
county_data[county_data.county == "New York City"].query("date=={}".format("'" + latestDate + "'"))

,date,county,state,fips,cases,deaths
352207,2020-07-20,New York City,New York,NaN,226388,22882
352207,2020-07-20,New York City,New York,36061,226388,22882
352207,2020-07-20,New York City,New York,36047,226388,22882
352207,2020-07-20,New York City,New York,36081,226388,22882
352207,2020-07-20,New York City,New York,36005,226388,22882
352207,2020-07-20,New York City,New York,36085,226388,22882
352207,2020-07-20,New York City,New York,36061,45277,4576
352207,2020-07-20,New York City,New York,36061,45277,4576
352207,2020-07-20,New York City,New York,36061,45277,4576
352207,2020-07-20,New York City,New York,36061,45277,4576


In [38]:
county_data[county_data.fips.isna()][county_data.county == "Unknown"].head()

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,date,county,state,fips,cases,deaths
418,2020-03-01,Unknown,Rhode Island,NaN,2,0
450,2020-03-02,Unknown,Rhode Island,NaN,2,0
485,2020-03-03,Unknown,Rhode Island,NaN,2,0
522,2020-03-04,Unknown,Rhode Island,NaN,2,0
569,2020-03-05,Unknown,Rhode Island,NaN,2,0


In [11]:
fips = pd.DataFrame(counties['features'])

In [12]:
len(fips.id)

3221

In [14]:
county_data.fips.nunique()

3176

In [15]:
3221-3176

45

In [123]:
# CALCULATED VALUES

latestDate = county_data.date.max()
latest_dt = datetime.strptime(latestDate, '%Y-%m-%d')
days_since_outbreak = (latest_dt - datetime.strptime('2019-12-31', '%Y-%m-%d')).days
days_since_USA = (latest_dt - datetime.strptime('2020-01-21', '%Y-%m-%d')).days
day_prior = datetime.strftime((latest_dt - timedelta(days=1)),'%Y-%m-%d')
two_prior = datetime.strftime((latest_dt - timedelta(days=2)),'%Y-%m-%d')

confirmed_cases = county_data.query("date=={}".format("'" + latestDate + "'")).cases.sum()
cases_day_prior = county_data.query("date=={}".format("'" + day_prior + "'")).cases.sum()
cases_two_prior = county_data.query("date=={}".format("'" + two_prior + "'")).cases.sum()

case_rate = (confirmed_cases - cases_day_prior) / cases_day_prior
case_prior_rate = (cases_day_prior - cases_two_prior) / cases_two_prior
case_rate_delta = (case_rate - case_prior_rate) / case_prior_rate
if case_rate_delta < 0:
    case_rate_delta_s = '{:.2%}'.format(case_rate_delta)
else:
    case_rate_delta_s = '+ {:.2%}'.format(case_rate_delta)
    
case_delta = confirmed_cases - cases_day_prior
case_percent_diff = (case_delta) / cases_day_prior


confirmed_deaths = county_data.query("date=={}".format("'" + latestDate + "'")).deaths.sum()
death_day_prior = county_data.query("date=={}".format("'" + day_prior + "'")).deaths.sum()
death_two_prior = county_data.query("date=={}".format("'" + two_prior + "'")).deaths.sum()

death_rate = (confirmed_deaths - death_day_prior) / death_day_prior
death_prior_rate = (death_day_prior - death_two_prior) / death_two_prior
death_rate_delta = (death_rate - death_prior_rate) / death_prior_rate
if death_rate_delta < 0:
    death_rate_delta_s = '{:.2%}'.format(death_rate_delta)
else:
    death_rate_delta_s = '+ {:.2%}'.format(death_rate_delta)
    

death_delta = confirmed_deaths - death_day_prior
death_percent_diff = (death_delta) / death_day_prior

latest = county_data.query("date=={}".format("'" + latestDate + "'"))


total = state_data.groupby(state_data.date).sum()
difference = total.diff()
difference = difference.fillna(0)
difference.columns = ['case_increase','death_increase']
new = total.merge(difference, left_index=True, right_index=True)


In [96]:
latest

,date,county,state,fips,cases,deaths
350365,2020-07-20,Autauga,Alabama,01001,857,21
350366,2020-07-20,Baldwin,Alabama,01003,2013,15
350367,2020-07-20,Barbour,Alabama,01005,503,4
350368,2020-07-20,Bibb,Alabama,01007,279,2
350369,2020-07-20,Blount,Alabama,01009,507,1
350370,2020-07-20,Bullock,Alabama,01011,400,11
350371,2020-07-20,Butler,Alabama,01013,689,33
350372,2020-07-20,Calhoun,Alabama,01015,832,6
350373,2020-07-20,Chambers,Alabama,01017,767,33
350374,2020-07-20,Cherokee,Alabama,01019,179,7


In [18]:
latest.head()

,date,county,state,fips,cases,deaths
350365,2020-07-20,Autauga,Alabama,01001,857,21
350366,2020-07-20,Baldwin,Alabama,01003,2013,15
350367,2020-07-20,Barbour,Alabama,01005,503,4
350368,2020-07-20,Bibb,Alabama,01007,279,2
350369,2020-07-20,Blount,Alabama,01009,507,1


In [124]:


census = pd.read_csv('data/co-est2019-alldata.csv',encoding = 'latin-1',dtype={"STATE": str,"COUNTY":str})
census = census.query('COUNTY!="000"')
census["fips"] = census.STATE + census.COUNTY
pop = census[['POPESTIMATE2019']].copy()
pop.columns = ['Population']
pop.index = census.fips
county_pop = pop.to_dict()

In [125]:

county_data = county_data.merge(pop, how='outer',on='fips')

county_data['percent_pop'] = round(10*county_data.cases/county_data.Population,2)
latest2 = county_data.query("date=={}".format("'" + latestDate + "'"))
top = latest2.percent_pop.max()

In [126]:
nyc = latest2[latest2.county == "New York City"]

In [130]:
county_data.fips

0         53061
1         53061
2         53061
3         53061
4         53061
5         53061
6         53061
7         53061
8         53061
9         53061
10        53061
11        53061
12        53061
13        53061
14        53061
15        53061
16        53061
17        53061
18        53061
19        53061
20        53061
21        53061
22        53061
23        53061
24        53061
25        53061
26        53061
27        53061
28        53061
29        53061
          ...  
354297    30077
354298    30079
354299    31005
354300    31009
354301    31017
354302    31075
354303    31085
354304    31103
354305    31113
354306    31115
354307    31183
354308    32009
354309    35011
354310    36005
354311    36047
354312    36061
354313    36081
354314    36085
354315    38001
354316    41069
354317    46063
354318    46107
354319    48033
354320    48269
354321    48301
354322    48431
354323    49009
354324    49055
354325    51690
354326    54017
Name: fips, Length: 3543

In [46]:
burroughs = ['New York', 'Kings', 'Queens', 'Bronx', 'Richmond']

In [71]:
burrs = [36061, 36047, 36081, 36005, 36085]
nyc_pop = 8400000

In [72]:
for i,each in enumerate(burrs):
    extra = [each, nyc_pop, '2020-07-20','New York City','New York',226388,22882,0]
    latest2.loc[len(latest2)+i] = [i] + extra

ValueError: cannot set a row with mismatched columns

In [62]:
extra

[36085, 8400000, '2020-07-20', 'New York City', 'New York', 226388, 22882, 0]

In [63]:
latest2[latest2.fips == 36061]

,0,Population,cases,county,date,deaths,fips,percent_pop,state,1,2,3,4,5,6,7


In [103]:
a = {1:'a'}

In [104]:
a[1]

'a'